In [3]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline   # tfid와 nb를 묶어서 실행해줌
from sklearn.metrics import classification_report

In [7]:
# 데이터 로드
article_df = pd.read_csv("article.csv")
len(article_df)

600

In [8]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
    article_df.content, article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(540, 60, 540, 60)

In [9]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

In [10]:
# 학습
model = clf.fit(X_train.values.astype("str"), y_train)

In [11]:
# 모델 성능 확인
y_pred = model.predict(X_test)

In [14]:
list(y_test[:5]), list(y_pred[:5])

([104, 104, 105, 104, 102], [104, 104, 105, 104, 102])

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       1.00      1.00      1.00        10
         101       1.00      1.00      1.00         8
         102       1.00      1.00      1.00        10
         103       1.00      1.00      1.00         7
         104       1.00      1.00      1.00        16
         105       1.00      1.00      1.00         9

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



### 모델 사용하기

In [18]:
categories = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

In [22]:
contents = [
    "네이버와 카카오 금융산업 진출",
    "광화문 대한애국당 천막, 격렬 저항 속 철거",
    "요즘 환율 주가 예측이 불가",
]


In [23]:
datas = {
    "content": contents,
    "category_code": model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,네이버와 카카오 금융산업 진출,105
1,"광화문 대한애국당 천막, 격렬 저항 속 철거",103
2,요즘 환율 주가 예측이 불가,103


In [36]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(
    lambda data: round(max(model.predict_proba([data])[0]), 2))
df

,content,category_code,category,proba
0,네이버와 카카오 금융산업 진출,105,IT/과학,0.77
1,"광화문 대한애국당 천막, 격렬 저항 속 철거",103,생활/문화,0.17
2,요즘 환율 주가 예측이 불가,103,생활/문화,0.17


In [29]:
 model.predict_proba([contents[0]])

array([[0.0378369 , 0.03899978, 0.05290765, 0.04235109, 0.05583531,
        0.77206927]])

In [30]:
round(max(model.predict_proba([contents[0]])[0]), 2)

0.77

In [37]:
# 모델저장

In [38]:
pickle.dump(model, open("clf.pkl", "wb"))